## Chains ⛓️⛓️⛓️
Combining different LLM calls and action automatically

Ex: Summary #1, Summary #2, Summary #3 > Final Summary

Check out [this video](https://www.youtube.com/watch?v=f9_BWhCI4Zo&t=2s) explaining different summarization chain types

There are [many applications of chains](https://python.langchain.com/en/latest/modules/chains/how_to_guides.html) search to see which are best for your use case.

We'll cover two of them:

### 1. Simple Sequential Chains
Easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks (and keeping your LLM focused)

In [1]:
import os

from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

In [2]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = prompt_template | llm

In [3]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

/Users/yanivbenzvi/develop/tikal/langchain-tutorial/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
chains = location_chain.pipe(meal_chain)

In [5]:
chains.invoke(input={"user_location": "rome"})

{'user_meal': 'One classic dish from Rome is Cacio e Pepe. Made with just three simple ingredients - pasta, cheese, and black pepper - this dish has been a staple in Roman cuisine for centuries. The pasta is cooked until al dente and then tossed with a creamy sauce made from the cheese and pasta water. The freshly ground black pepper adds a unique spicy kick to the dish. Cacio e Pepe is a simple yet delicious representation of the traditional flavors of Rome.',
 'text': 'To make Cacio e Pepe at home, bring a pot of salted water to a boil and cook your choice of pasta until al dente. While the pasta is cooking, finely grate Pecorino Romano cheese. In a separate pan, melt butter and slowly add in pasta water until a creamy consistency is achieved. Once the pasta is cooked, drain and add it to the pan with the butter and water mixture. Slowly stir in the grated cheese until it melts and creates a smooth sauce. Serve the pasta with a generous amount of freshly ground black pepper on top. E

In [6]:
# pipe operator is implemented by __or__ magic method in runnable class
chains = location_chain | meal_chain

In [7]:
result = chains.invoke(input={"user_location": "rome"})
result

{'user_meal': 'One classic dish from Rome is spaghetti alla carbonara, a dish that originated in the Lazio region. This dish is made with spaghetti pasta, egg, pecorino cheese, guanciale (cured pork jowl), and black pepper. The name "carbonara" comes from the Italian word for coal, referring to the black pepper that gives the dish its characteristic specks. The dish is popular among locals and tourists alike and is a staple in many Italian restaurants. It is believed that the dish was first created as a hearty meal for Italian charcoal workers, and it has since become a beloved Roman dish worldwide. ',
 'text': ' \nTo make spaghetti alla carbonara at home, start by boiling a pot of salted water and cooking the spaghetti until al dente. While the pasta is cooking, fry slices of guanciale in a pan until crispy. In a separate bowl, beat eggs and add grated pecorino cheese. Once the spaghetti is cooked, drain it and add it to the pan with the guanciale. Turn off the heat and quickly add th

In [8]:
result.get('user_meal')

'One classic dish from Rome is spaghetti alla carbonara, a dish that originated in the Lazio region. This dish is made with spaghetti pasta, egg, pecorino cheese, guanciale (cured pork jowl), and black pepper. The name "carbonara" comes from the Italian word for coal, referring to the black pepper that gives the dish its characteristic specks. The dish is popular among locals and tourists alike and is a staple in many Italian restaurants. It is believed that the dish was first created as a hearty meal for Italian charcoal workers, and it has since become a beloved Roman dish worldwide. '

In [9]:
print(result.get('text'))

 
To make spaghetti alla carbonara at home, start by boiling a pot of salted water and cooking the spaghetti until al dente. While the pasta is cooking, fry slices of guanciale in a pan until crispy. In a separate bowl, beat eggs and add grated pecorino cheese. Once the spaghetti is cooked, drain it and add it to the pan with the guanciale. Turn off the heat and quickly add the egg and cheese mixture, stirring it into the spaghetti until it becomes creamy. Add black pepper to taste and serve hot, garnished with extra cheese and parsley, if desired. Enjoy your homemade spaghetti alla carbonara!


### branching chains
In LangChain v0.1 of langchain, we introduced the ability to create branching chains using the `BranchingChain` class. This allows you to create a chain that can take multiple paths based on the output of a previous chain.